## metric


In [1]:
import json

import numba
import numpy as np
from numba import types
import numpy.typing as npt
import pandas as pd
import scipy.optimize


class ParticipantVisibleError(Exception):
    pass


@numba.jit(nopython=True)
def _rle_encode_jit(x: npt.NDArray, fg_val: int = 1) -> list[int]|str:
    """Numba-jitted RLE encoder."""
    dots = np.where(x.T.flatten() == fg_val)[0]
    if len(dots) == 0:
        return "authentic"
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def rle_encode(masks: list[npt.NDArray], fg_val: int = 1) -> str:
    """
    Adapted from contrails RLE https://www.kaggle.com/code/inversion/contrails-rle-submission
    Args:
        masks: list of numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encodings as a string, with each RLE JSON-encoded and separated by a semicolon.
    """
    return ';'.join([json.dumps(_rle_encode_jit(x, fg_val)) for x in masks])


@numba.njit
def _rle_decode_jit(mask_rle: npt.NDArray, height: int, width: int) -> npt.NDArray:
    """
    s: numpy array of run-length encoding pairs (start, length)
    shape: (height, width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    """
    if len(mask_rle) % 2 != 0:
        # Numba requires raising a standard exception.
        raise ValueError('One or more rows has an odd number of values.')

    starts, lengths = mask_rle[0::2], mask_rle[1::2]
    starts -= 1
    ends = starts + lengths
    for i in range(len(starts) - 1):
        if ends[i] > starts[i + 1]:
            raise ValueError('Pixels must not be overlapping.')
    img = np.zeros(height * width, dtype=np.bool_)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img


def rle_decode(mask_rle: str, shape: tuple[int, int]) -> npt.NDArray:
    """
    mask_rle: run-length as string formatted (start length)
              empty predictions need to be encoded with '-'
    shape: (height, width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    """

    mask_rle = json.loads(mask_rle)
    mask_rle = np.asarray(mask_rle, dtype=np.int32)
    starts = mask_rle[0::2]
    if sorted(starts) != list(starts):
        raise ParticipantVisibleError('Submitted values must be in ascending order.')
    try:
        return _rle_decode_jit(mask_rle, shape[0], shape[1]).reshape(shape, order='F')
    except ValueError as e:
        raise ParticipantVisibleError(str(e)) from e


def calculate_f1_score(pred_mask: npt.NDArray, gt_mask: npt.NDArray):
    pred_flat = pred_mask.flatten()
    gt_flat = gt_mask.flatten()

    tp = np.sum((pred_flat == 1) & (gt_flat == 1))
    fp = np.sum((pred_flat == 1) & (gt_flat == 0))
    fn = np.sum((pred_flat == 0) & (gt_flat == 1))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    if (precision + recall) > 0:
        return 2 * (precision * recall) / (precision + recall)
    else:
        return 0


def calculate_f1_matrix(pred_masks: list[npt.NDArray], gt_masks: list[npt.NDArray]):
    """
    Parameters:
    pred_masks (np.ndarray):
            First dimension is the number of predicted instances.
            Each instance is a binary mask of shape (height, width).
    gt_masks (np.ndarray):
            First dimension is the number of ground truth instances.
            Each instance is a binary mask of shape (height, width).
    """

    num_instances_pred = len(pred_masks)
    num_instances_gt = len(gt_masks)
    f1_matrix = np.zeros((num_instances_pred, num_instances_gt))

    # Calculate F1 scores for each pair of predicted and ground truth masks
    for i in range(num_instances_pred):
        for j in range(num_instances_gt):
            pred_flat = pred_masks[i].flatten()
            gt_flat = gt_masks[j].flatten()
            f1_matrix[i, j] = calculate_f1_score(pred_mask=pred_flat, gt_mask=gt_flat)

    if f1_matrix.shape[0] < len(gt_masks):
        # Add a row of zeros to the matrix if the number of predicted instances is less than ground truth instances
        f1_matrix = np.vstack((f1_matrix, np.zeros((len(gt_masks) - len(f1_matrix), num_instances_gt))))

    return f1_matrix


def oF1_score(pred_masks: list[npt.NDArray], gt_masks: list[npt.NDArray]):
    """
    Calculate the optimal F1 score for a set of predicted masks against
    ground truth masks which considers the optimal F1 score matching.
    This function uses the Hungarian algorithm to find the optimal assignment
    of predicted masks to ground truth masks based on the F1 score matrix.
    If the number of predicted masks is less than the number of ground truth masks,
    it will add a row of zeros to the F1 score matrix to ensure that the dimensions match.

    Parameters:
    pred_masks (list of np.ndarray): List of predicted binary masks.
    gt_masks (np.ndarray): Array of ground truth binary masks.
    Returns:
    float: Optimal F1 score.
    """
    f1_matrix = calculate_f1_matrix(pred_masks, gt_masks)

    # Find the best matching between predicted and ground truth masks
    row_ind, col_ind = scipy.optimize.linear_sum_assignment(-f1_matrix)
    # The linear_sum_assignment discards excess predictions so we need a separate penalty.
    excess_predictions_penalty = len(gt_masks) / max(len(pred_masks), len(gt_masks))
    return np.mean(f1_matrix[row_ind, col_ind]) * excess_predictions_penalty


def evaluate_single_image(label_rles: str, prediction_rles: str, shape_str: str) -> float:
    shape = json.loads(shape_str)
    label_rles = [rle_decode(x, shape=shape) for x in label_rles.split(';')]
    prediction_rles = [rle_decode(x, shape=shape) for x in prediction_rles.split(';')]
    return oF1_score(prediction_rles, label_rles)


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Args:
        solution (pd.DataFrame): The ground truth DataFrame.
        submission (pd.DataFrame): The submission DataFrame.
        row_id_column_name (str): The name of the column containing row IDs.
    Returns:
        float

    Examples
    --------
    >>> solution = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['authentic', 'authentic', 'authentic'], 'shape': ['authentic', 'authentic', 'authentic']})
    >>> submission = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['authentic', 'authentic', 'authentic']})
    >>> score(solution.copy(), submission.copy(), row_id_column_name='row_id')
    1.0

    >>> solution = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['authentic', 'authentic', 'authentic'], 'shape': ['authentic', 'authentic', 'authentic']})
    >>> submission = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['[101, 102]', '[101, 102]', '[101, 102]']})
    >>> score(solution.copy(), submission.copy(), row_id_column_name='row_id')
    0.0

    >>> solution = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['[101, 102]', '[101, 102]', '[101, 102]'], 'shape': ['[720, 960]', '[720, 960]', '[720, 960]']})
    >>> submission = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['[101, 102]', '[101, 102]', '[101, 102]']})
    >>> score(solution.copy(), submission.copy(), row_id_column_name='row_id')
    1.0

    >>> solution = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['[101, 103]', '[101, 102]', '[101, 102]'], 'shape': ['[720, 960]', '[720, 960]', '[720, 960]']})
    >>> submission = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['[101, 102]', '[101, 102]', '[101, 102]']})
    >>> score(solution.copy(), submission.copy(), row_id_column_name='row_id')
    0.9983739837398374

    >>> solution = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['[101, 102];[300, 100]', '[101, 102]', '[101, 102]'], 'shape': ['[720, 960]', '[720, 960]', '[720, 960]']})
    >>> submission = pd.DataFrame({'row_id': [0, 1, 2], 'annotation': ['[101, 102]', '[101, 102]', '[101, 102]']})
    >>> score(solution.copy(), submission.copy(), row_id_column_name='row_id')
    0.8333333333333334
    """
    df = solution
    df = df.rename(columns={'annotation': 'label'})

    df['prediction'] = submission['annotation']
    # Check for correct 'authentic' label
    authentic_indices = (df['label'] == 'authentic') | (df['prediction'] == 'authentic')
    df['image_score'] = ((df['label'] == df['prediction']) & authentic_indices).astype(float)

    df.loc[~authentic_indices, 'image_score'] = df.loc[~authentic_indices].apply(
        lambda row: evaluate_single_image(row['label'], row['prediction'], row['shape']), axis=1
    )
    return float(np.mean(df['image_score']))


In [2]:
# --- Transformer-based DinoMFDecoder (Mask2Former-style, single-scale minimal) ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple

class _MaskDecoderLayer(nn.Module):
    """
    One decoder stage composed of CA (cross-attention), SA (self-attention), and FFN.
    - CA: queries attend to pixel tokens (image features)
    - SA: queries attend to themselves to resolve mutual relations
    - FFN: position-wise feed-forward for non-linearity
    Order here follows the diagram: CA -> SA -> FFN.
    """
    def __init__(self, d_model: int, nhead: int, dim_feedforward: int = 1024, dropout: float = 0.1):
        super().__init__()
        self.cross_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.self_attn  = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.linear1    = nn.Linear(d_model, dim_feedforward)
        self.linear2    = nn.Linear(dim_feedforward, d_model)
        self.dropout    = nn.Dropout(dropout)
        self.dropout1   = nn.Dropout(dropout)
        self.dropout2   = nn.Dropout(dropout)
        self.norm1      = nn.LayerNorm(d_model)
        self.norm2      = nn.LayerNorm(d_model)
        self.norm3      = nn.LayerNorm(d_model)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, q: torch.Tensor, kv: torch.Tensor, kv_pos: torch.Tensor = None):
        # Cross-Attention (queries <- pixels)
        k = v = kv if kv_pos is None else kv + kv_pos
        q2, _ = self.cross_attn(query=q, key=k, value=v, need_weights=False)
        q = q + self.dropout1(q2)
        q = self.norm1(q)
        # Self-Attention (queries <- queries)
        q2, _ = self.self_attn(query=q, key=q, value=q, need_weights=False)
        q = q + self.dropout2(q2)
        q = self.norm2(q)
        # FFN
        q2 = self.linear2(self.dropout(self.activation(self.linear1(q))))
        q = self.norm3(q + q2)
        return q

class DinoMFDecoder(nn.Module):
    """
    Mask2Former-like minimal decoder:
    - Projects DINOv2 fmap (B,C,h,w) -> (B,D,h,w) as pixel embeddings
    - Uses learnable queries (Q,D)
    - L stages of [CA -> SA -> FFN]
    - Produces Q masks via dot-product between query mask-embeds and pixel embeddings
    Returns a single-channel mask by max-pooling across queries (compatible with existing pipeline).
    """
    def __init__(
        self,
        in_ch: int = 768,
        out_ch: int = 1,
        d_model: int = 256,
        nhead: int = 8,
        num_queries: int = 64,
        num_layers: int = 2,
        dim_feedforward: int = 1024,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.d_model = d_model
        self.pixel_proj = nn.Conv2d(in_ch, d_model, kernel_size=1)
        self.query_embed = nn.Embedding(num_queries, d_model)
        self.layers = nn.ModuleList([
            _MaskDecoderLayer(d_model, nhead, dim_feedforward, dropout) for _ in range(num_layers)
        ])
        self.mask_embed = nn.Linear(d_model, d_model)
        self.class_embed = nn.Linear(d_model, 2)  # forged / void (optional)
        # simple 2D sine-cos positional encoding for pixels
        self.register_buffer("pe_cache", None, persistent=False)

    def _pos2d(self, H: int, W: int, device) -> torch.Tensor:
        # cached for a given HxW
        if self.pe_cache is not None and self.pe_cache.shape[1] == H*W:
            return self.pe_cache
        y, x = torch.meshgrid(torch.arange(H, device=device), torch.arange(W, device=device), indexing="ij")
        omega = torch.arange(self.d_model // 4, device=device, dtype=torch.float32) / (self.d_model // 4)
        omega = 1.0 / (10000 ** omega)
        out_y = torch.cat([torch.sin(y.reshape(-1,1) * omega), torch.cos(y.reshape(-1,1) * omega)], dim=1)
        out_x = torch.cat([torch.sin(x.reshape(-1,1) * omega), torch.cos(x.reshape(-1,1) * omega)], dim=1)
        pe = torch.cat([out_y, out_x], dim=1)
        if pe.shape[1] < self.d_model:
            pe = F.pad(pe, (0, self.d_model - pe.shape[1]))
        else:
            pe = pe[:, :self.d_model]
        pe = pe.unsqueeze(0)  # (1, HW, D)
        self.pe_cache = pe
        return pe

    def forward(self, f: torch.Tensor, target_size: Tuple[int, int]):
        # f: (B, C, h, w)
        B, C, h, w = f.shape
        device = f.device
        pix = self.pixel_proj(f)                 # (B, D, h, w)
        pix_flat = pix.flatten(2).permute(0, 2, 1)  # (B, HW, D)
        pos = self._pos2d(h, w, device).expand(B, -1, -1)  # (B, HW, D)

        # prepare queries
        q = self.query_embed.weight.unsqueeze(0).expand(B, -1, -1)  # (B, Q, D)
        for layer in self.layers:
            q = layer(q, kv=pix_flat, kv_pos=pos)

        # class logits (optional, not used downstream yet)
        cls_logits = self.class_embed(q)  # (B, Q, 2)

        # mask logits via dot product between query mask-embed and pixel embeddings
        mask_tokens = self.mask_embed(q)              # (B, Q, D)
        pix_grid = pix                               # (B, D, h, w)
        mask_logits = torch.einsum('bqd,bdhw->bqhw', mask_tokens, pix_grid)  # (B, Q, h, w)

        # collapse queries by max (keep old interface: out_ch=1)
        logits_1 = mask_logits.max(dim=1, keepdim=True).values  # (B,1,h,w)
        logits_1 = F.interpolate(logits_1, size=target_size, mode='bilinear', align_corners=False)
        return logits_1

# Usage note:
# - To switch: model_seg.seg_head = DinoMFDecoder(768, 1, d_model=256, nhead=8, num_queries=64, num_layers=2)
# - This keeps the same forward_seg signature and downstream pipeline.


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os, cv2, json, math, random, torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from transformers import AutoImageProcessor, AutoModel
from metric import *

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # This forces CUDA to use deterministic algorithms (slower but consistent)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Environment detection & dynamic paths (Kaggle vs Colab) ---
IS_KAGGLE = os.path.exists('/kaggle/input/recodai-luc-scientific-image-forgery-detection')
IS_COLAB = 'google.colab' in globals() or 'COLAB_GPU' in os.environ

BASE_DIR  = "/kaggle/input/recodai-luc-scientific-image-forgery-detection" if IS_KAGGLE else "/content/input/"

AUTH_DIR  = f"{BASE_DIR}/train_images/authentic"
FORG_DIR  = f"{BASE_DIR}/train_images/forged"
MASK_DIR  = f"{BASE_DIR}/train_masks"
TEST_DIR  = f"{BASE_DIR}/test_images"

# DINO model reference: Kaggle dataset path or Hugging Face ID for Colab
DINO_PATH = "/kaggle/input/dinov2/pytorch/base/1" if IS_KAGGLE else os.environ.get("DINO_ID", "facebook/dinov2-base")

# On Kaggle, prefer local files only; on Colab, allow remote download
USE_LOCAL_ONLY = True if IS_KAGGLE else False

IMG_SIZE = 512
BATCH_SIZE = 2
# MODEL_LOC examples for Kaggle (optional). In Colab, set to None or a local path under /content.
# MODEL_LOC = '/kaggle/input/cnndinov2-pbd/CNNDINOv2-U54/CNNDINOv2-U54/model_seg_final.pt'  # 0.310
# MODEL_LOC = '/kaggle/input/cnndinov2-pbd/CNNDINOv2-U52/CNNDINOv2-U52/model_seg_final.pt'  # 0.321
MODEL_LOC = '/kaggle/input/dinov2-mfdecorder/pytorch/v1/1/model_seg_final.pt' if IS_KAGGLE else None
EPOCHS_SEG = 4
LR_SEG = 1e-5
WEIGHT_DECAY = 2e-5
VAL_SAMPLES = 10 # validate on a small forged subset per epoch

# INFERENCE UTILS
AREA_THR = 200
MEAN_THR = 0.22
USE_TTA = True
GRID_SEARCH = False

class ForgerySegDataset(Dataset):
    def __init__(self, auth_paths, forg_paths, mask_dir, img_size=IMG_SIZE):
        self.samples = []
        for p in forg_paths:
            m = os.path.join(mask_dir, Path(p).stem + ".npy")
            if os.path.exists(m):
                self.samples.append((p, m))
        for p in auth_paths:
            self.samples.append((p, None))
        self.img_size = img_size
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img_path, mask_path = self.samples[idx]
        img = Image.open(img_path).convert("RGB")
        w, h = img.size
        if mask_path is None:
            mask = np.zeros((h, w), np.uint8)
        else:
            m = np.load(mask_path)
            if m.ndim == 3: m = np.max(m, axis=0)
            mask = (m > 0).astype(np.uint8)
        img_r = img.resize((IMG_SIZE, IMG_SIZE))
        mask_r = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        img_t = torch.from_numpy(np.array(img_r, np.float32)/255.).permute(2,0,1)
        mask_t = torch.from_numpy(mask_r[None, ...].astype(np.float32))
        return img_t, mask_t


#  MODEL (DINOv2 + Decoder)

processor = AutoImageProcessor.from_pretrained(DINO_PATH, local_files_only=USE_LOCAL_ONLY, use_fast=False)
encoder = AutoModel.from_pretrained(DINO_PATH, local_files_only=USE_LOCAL_ONLY).eval().to(device)

class DinoTinyDecoder(nn.Module):
    def __init__(self, in_ch=768, out_ch=1):
        super().__init__()
        # Block 1: 768 -> 384
        self.block1 = nn.Sequential(
            nn.Conv2d(in_ch, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1)
        )
        # Block 2: 384 -> 192
        self.block2 = nn.Sequential(
            nn.Conv2d(384, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1)
        )
        # Block 3: 192 -> 96
        self.block3 = nn.Sequential(
            nn.Conv2d(192, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        # Final Output: 96 -> 1
        self.conv_out = nn.Conv2d(96, out_ch, kernel_size=1)

    def forward(self, f, target_size):
        # f: [B, 768, 37, 37]

        # Step 1: Up to ~74x74
        x = F.interpolate(self.block1(f), size=(74, 74), mode='bilinear', align_corners=False)

        # Step 2: Up to ~148x148
        x = F.interpolate(self.block2(x), size=(148, 148), mode='bilinear', align_corners=False)

        # Step 3: Up to ~296x296
        x = F.interpolate(self.block3(x), size=(296, 296), mode='bilinear', align_corners=False)

        # Step 4: Final jump to 518x518
        x = self.conv_out(x)
        x = F.interpolate(x, size=target_size, mode='bilinear', align_corners=False)

        return x

class DinoSegmenter(nn.Module):
    def __init__(self, encoder, processor):
        super().__init__()
        self.encoder, self.processor = encoder, processor
        for p in self.encoder.parameters(): p.requires_grad = False
        # self.seg_head = DinoMFDecoder(768,1)
        self.seg_head = DinoTinyDecoder(768,1)

    def forward_features(self,x):
        imgs = (x*255).clamp(0,255).byte().permute(0,2,3,1).cpu().numpy()
        inputs = self.processor(images=list(imgs), return_tensors="pt").to(x.device)
        # with torch.no_grad():
        #     feats = self.encoder(**inputs).last_hidden_state
        feats = self.encoder(**inputs).last_hidden_state
        B,N,C = feats.shape
        fmap = feats[:,1:,:].permute(0,2,1)
        s = int(math.sqrt(N-1))
        fmap = fmap.reshape(B,C,s,s)
        return fmap
    def forward_seg(self,x):
        fmap = self.forward_features(x)
        return self.seg_head(fmap,(IMG_SIZE,IMG_SIZE))

# --- 3.1 Utilities: init bias, losses, small eval ---

def init_last_conv_bias_negative(module: nn.Module, bias_value: float = -4.0):
    """Initialize the final 1-channel Conv2d bias to a negative value to avoid early saturation."""
    for m in module.modules():
        if isinstance(m, nn.Conv2d) and m.out_channels == 1:
            if m.bias is not None:
                nn.init.constant_(m.bias, bias_value)

class SoftDiceLoss(nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
    def forward(self, logits: torch.Tensor, targets: torch.Tensor):
        probs = torch.sigmoid(logits)
        targets = targets.float()
        inter = (probs * targets).sum(dim=(2,3))
        denom = probs.sum(dim=(2,3)) + targets.sum(dim=(2,3)) + self.eps
        dice = (2 * inter + self.eps) / denom
        return 1 - dice.mean()

@torch.no_grad()
def evaluate_mean_f1(model_seg, val_list, mask_dir, max_items=VAL_SAMPLES):
    model_seg.eval()
    items = val_list[:max_items]
    f1s = []
    for p in items:
        pil = Image.open(p).convert("RGB")
        x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)
        logits = model_seg.forward_seg(x)
        prob = torch.sigmoid(logits)[0,0].cpu().numpy()
        # simple mask (same as pipeline)
        gx = cv2.Sobel(prob, cv2.CV_32F, 1, 0, ksize=3)
        gy = cv2.Sobel(prob, cv2.CV_32F, 0, 1, ksize=3)
        grad_mag = np.sqrt(gx**2 + gy**2)
        grad_norm = grad_mag / (grad_mag.max() + 1e-6)
        enhanced = 0.65 * prob + 0.35 * grad_norm
        enhanced = cv2.GaussianBlur(enhanced, (3,3), 0)
        thr = np.mean(enhanced) + 0.3 * np.std(enhanced)
        mask = (enhanced > thr).astype(np.uint8)
        m_gt = np.load(Path(mask_dir)/f"{Path(p).stem}.npy")
        if m_gt.ndim == 3: m_gt = np.max(m_gt, axis=0)
        m_gt = (m_gt > 0).astype(np.uint8)
        # align to original size
        mask = cv2.resize(mask, (m_gt.shape[1], m_gt.shape[0]), interpolation=cv2.INTER_NEAREST)
        f1s.append(f1_score(m_gt.flatten(), mask.flatten(), zero_division=0))
    return float(np.mean(f1s)) if f1s else 0.0


# --- 4. DATA SPLIT AND TRAINING ---
auth_imgs = sorted([str(Path(AUTH_DIR)/f) for f in os.listdir(AUTH_DIR)])
forg_imgs = sorted([str(Path(FORG_DIR)/f) for f in os.listdir(FORG_DIR)])
train_auth, val_auth = train_test_split(auth_imgs, test_size=0.2, random_state=42)
train_forg, val_forg = train_test_split(forg_imgs, test_size=0.2, random_state=42)

train_loader = DataLoader(ForgerySegDataset(train_auth, train_forg, MASK_DIR),
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(ForgerySegDataset(val_auth, val_forg, MASK_DIR),
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_seg = DinoSegmenter(encoder, processor).to(device)

init_last_conv_bias_negative(model_seg.seg_head, bias_value=-4.0)

# Train only decoder
opt_seg = optim.AdamW(model_seg.seg_head.parameters(), lr=LR_SEG, weight_decay=WEIGHT_DECAY)
crit_bce = nn.BCEWithLogitsLoss()
crit_dice = SoftDiceLoss()

def seg_loss(logits, targets, alpha=0.5):
    return alpha * crit_bce(logits, targets) + (1 - alpha) * crit_dice(logits, targets)

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

def train_seg(epoch):
    best_f1 = -1.0
    best_path = "model_seg_best.pt"

    for e in range(epoch):
        model_seg.train()
        model_seg.encoder.eval()  # keep encoder frozen/eval
        total_loss = 0.0

        for x, m in tqdm(train_loader, desc=f"[Segmentation] Epoch {e+1}/{epoch}"):
            x, m = x.to(device), m.to(device)
            opt_seg.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model_seg.forward_seg(x)
                loss = seg_loss(logits, m)
            scaler.scale(loss).backward()
            scaler.step(opt_seg)
            scaler.update()
            total_loss += loss.item()

        avg_loss = total_loss / max(1, len(train_loader))

        # quick validation on forged subset
        val_f1 = evaluate_mean_f1(model_seg, val_forg, MASK_DIR, max_items=VAL_SAMPLES)
        print(f"  → avg_loss={avg_loss:.4f} | val_f1@{VAL_SAMPLES}={val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model_seg.state_dict(), best_path)
            print(f"  ✅ New best saved to {best_path}")

    # Save final model state
    torch.save(model_seg.state_dict(), "model_seg_final.pt")
    print("Model saved as model_seg_final.pt")


# Load pretrained weights if MODEL_LOC is specified
if MODEL_LOC is not None and os.path.exists(MODEL_LOC):
    model_seg.load_state_dict(torch.load(MODEL_LOC, map_location=device))
    print(f"✅ Loaded pretrained model from: {MODEL_LOC}")
    model_seg.eval()  # Set model to evaluation mode
else:
    print("model not found, training start!")
    train_seg(EPOCHS_SEG)

@torch.no_grad()
def segment_prob_map(pil):
    x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)
    prob = torch.sigmoid(model_seg.forward_seg(x))[0,0].cpu().numpy()
    return prob

@torch.no_grad()
def segment_prob_map_with_tta(pil):
    # 1. Preprocessing: Resize, Normalize, and move to Device
    x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)

    predictions = []

    # 2. Original Prediction
    pred_orig = torch.sigmoid(model_seg.forward_seg(x))
    predictions.append(pred_orig)

    # 3. Horizontal Flip TTA (dim 3)
    # Flip input -> Predict -> Flip output back
    pred_h = torch.sigmoid(model_seg.forward_seg(torch.flip(x, dims=[3])))
    predictions.append(torch.flip(pred_h, dims=[3]))

    # 4. Vertical Flip TTA (dim 2)
    # Flip input -> Predict -> Flip output back
    pred_v = torch.sigmoid(model_seg.forward_seg(torch.flip(x, dims=[2])))
    predictions.append(torch.flip(pred_v, dims=[2]))

    # 5. Average the predictions and format as numpy
    # We stack the 3 predictions and take the mean across the stack dimension (0)
    prob = torch.stack(predictions).mean(0)[0, 0].cpu().numpy()

    return prob

def enhanced_adaptive_mask(prob, alpha_grad=0.45):
    gx = cv2.Sobel(prob, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(prob, cv2.CV_32F, 0, 1, ksize=3)
    grad_mag = np.sqrt(gx**2 + gy**2)
    grad_norm = grad_mag / (grad_mag.max() + 1e-6)
    enhanced = (1 - alpha_grad) * prob + alpha_grad * grad_norm
    enhanced = cv2.GaussianBlur(enhanced, (3,3), 0)
    thr = np.mean(enhanced) + 0.3 * np.std(enhanced)
    mask = (enhanced > thr).astype(np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    return mask, thr

def finalize_mask(prob, orig_size):
    mask, thr = enhanced_adaptive_mask(prob)
    mask = cv2.resize(mask, orig_size, interpolation=cv2.INTER_NEAREST)
    return mask, thr

def pipeline_final(pil):
    if USE_TTA:
        prob = segment_prob_map_with_tta(pil)
    else:
        prob = segment_prob_map(pil)
    mask, thr = finalize_mask(prob, pil.size)
    area = int(mask.sum())
    mean_inside = float(prob[cv2.resize(mask,(IMG_SIZE,IMG_SIZE),interpolation=cv2.INTER_NEAREST)==1].mean()) if area>0 else 0.0
    if area < AREA_THR or mean_inside < MEAN_THR:
        return "authentic", None, {"area": area, "mean_inside": mean_inside, "thr": thr}
    return "forged", mask, {"area": area, "mean_inside": mean_inside, "thr": thr}

import itertools

def grid_search_area_mean(forg_paths, auth_paths, mask_dir):
    mean_range = [round(x, 2) for x in np.arange(0.20, 0.291, 0.01)]
    area_range = [200]
    # 1. Use ALL images from both paths to maximize robustness
    val_set = [(p, "forged") for p in forg_paths] + [(p, "authentic") for p in auth_paths]

    print(f"🚀 Step 1: Caching probability maps for ALL {len(val_set)} images...")
    cache = []
    for p, label in tqdm(val_set):
        pil = Image.open(p).convert("RGB")
        w, h = pil.size

        # Get raw probability map
        prob = segment_prob_map_with_tta(pil) if USE_TTA else segment_prob_map(pil)

        # USE OLD MASK LOGIC: mean + 0.3*std
        mask_raw, _ = enhanced_adaptive_mask(prob) # Your function using np.mean + 0.3*np.std
        mask_resized = cv2.resize(mask_raw, (w, h), interpolation=cv2.INTER_NEAREST)

        # Handle Ground Truth
        if label == "forged":
            m_gt = np.load(Path(mask_dir)/f"{Path(p).stem}.npy")
            if m_gt.ndim == 3: m_gt = np.max(m_gt, axis=0)
            m_gt = (m_gt > 0).astype(np.uint8)
        else:
            m_gt = np.zeros((h, w), np.uint8) # Authentic = blank GT

        cache.append({"prob": prob, "mask": mask_resized, "gt": m_gt, "label": label})

    # 2. Sweep thresholds
    best_f1 = -1
    best_params = {}
    combinations = list(itertools.product(area_range, mean_range))

    for a_thr, m_thr in combinations:
        current_f1s = []
        for item in cache:
            mask = item["mask"]
            area = int(mask.sum()) # OLD AREA LOGIC

            # OLD MEAN LOGIC
            mask_small = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
            mean_in = float(item["prob"][mask_small == 1].mean()) if area > 0 else 0.0

            # Pipeline decision
            is_forged = (area >= a_thr and mean_in >= m_thr)
            m_pred = (mask > 0).astype(np.uint8) if is_forged else np.zeros_like(item["gt"])

            # F1 Calculation (Authentic silence = 1.0, noisy prediction = 0.0)
            f1 = f1_score(item["gt"].flatten(), m_pred.flatten(),
                          zero_division=1 if item["label"] == "authentic" else 0)
            current_f1s.append(f1)

        avg_f1 = np.mean(current_f1s)
        if avg_f1 > best_f1:
            best_f1 = avg_f1
            best_params = {"AREA_THR": a_thr, "MEAN_THR": m_thr}
            print(f"⭐ New Best F1: {best_f1:.4f} -> AREA: {a_thr}, MEAN: {m_thr}")

    return best_params

if GRID_SEARCH:
    best_cfg = grid_search_area_mean(val_forg, val_auth, MASK_DIR)
    AREA_THR = best_cfg['AREA_THR']
    MEAN_THR = best_cfg['MEAN_THR']


from sklearn.metrics import f1_score
val_items = [(p, 1) for p in val_forg[:10]]
results = []
for p,_ in tqdm(val_items, desc="Validation forged-only"):
    pil = Image.open(p).convert("RGB")
    label, m_pred, dbg = pipeline_final(pil)
    m_gt = np.load(Path(MASK_DIR)/f"{Path(p).stem}.npy")
    if m_gt.ndim==3: m_gt=np.max(m_gt,axis=0)
    m_gt=(m_gt>0).astype(np.uint8)
    m_pred=(m_pred>0).astype(np.uint8) if m_pred is not None else np.zeros_like(m_gt)
    f1 = f1_score(m_gt.flatten(), m_pred.flatten(), zero_division=0)
    results.append((Path(p).stem, f1, dbg))
print("\n F1-score par image falsifiée:\n")
for cid,f1,dbg in results:
    print(f"{cid} — F1={f1:.4f} | area={dbg['area']} mean={dbg['mean_inside']:.3f} thr={dbg['thr']:.3f}")
print(f"\n Moyenne F1 (falsifiées) = {np.mean([r[1] for r in results]):.4f}")


2026-01-10 07:37:51.420735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768030671.443099     131 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768030671.449711     131 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768030671.467503     131 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768030671.467530     131 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768030671.467533     131 computation_placer.cc:177] computation placer alr

AcceleratorError: CUDA error: uncorrectable ECC error encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def _is_valid_rle_item(s: str) -> bool:
    try:
        arr = json.loads(s)
        if not isinstance(arr, list):
            return False
        # Reject empty arrays explicitly
        if len(arr) == 0:
            return False
        # Must be even length (start,length pairs)
        if len(arr) % 2 != 0:
            return False
        starts = arr[0::2]
        lengths = arr[1::2]
        # Lengths must be positive
        if any(int(l) <= 0 for l in lengths):
            return False
        # Starts must be in ascending order (1-based indices expected)
        if sorted(starts) != list(starts):
            return False
        # Ensure no overlap within the same RLE sequence
        # Convert to 0-based starts for consistency with decoder
        starts0 = [int(s) - 1 for s in starts]
        ends0 = [s0 + int(l) for s0, l in zip(starts0, lengths)]
        for i in range(len(starts0) - 1):
            if ends0[i] > starts0[i + 1]:
                return False
        return True
    except Exception:
        return False

def sanitize_annotation_str(s: str) -> str:
    """
    Normalize a single annotation string:
    - If 'authentic': keep as-is
    - Else split by ';', drop empty/invalid JSON arrays, canonicalize JSON
    - If nothing left, return 'authentic'
    """
    s = str(s).strip()
    if s.lower() == 'authentic':
        return 'authentic'
    parts = [p for p in s.split(';') if p.strip()]
    keep = [json.dumps(json.loads(p)) for p in parts if _is_valid_rle_item(p)]
    return ';'.join(keep) if keep else 'authentic'

In [ ]:

import os, json, cv2
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- RLE Encoder for Kaggle Submission ---
# def rle_encode(mask: np.ndarray, fg_val: int = 1) -> str:
#     pixels = mask.T.flatten()
#     dots = np.where(pixels == fg_val)[0]
#     if len(dots) == 0:
#         return "authentic"
#     run_lengths = []
#     prev = -2
#     for b in dots:
#         if b > prev + 1:
#             run_lengths.extend((b + 1, 0))
#         run_lengths[-1] += 1
#         prev = b
#     return json.dumps([int(x) for x in run_lengths])

# --- Paths (dynamic with BASE_DIR from earlier cell) ---
TEST_DIR = f"{BASE_DIR}/test_images"
SAMPLE_SUB = f"{BASE_DIR}/sample_submission.csv"
OUT_PATH = "submission.csv"

rows = []
for f in tqdm(sorted(os.listdir(TEST_DIR)), desc="Inference on Test Set"):
    pil = Image.open(Path(TEST_DIR)/f).convert("RGB")
    label, mask, dbg = pipeline_final(pil)  # utilise la version améliorée

    # Sécurisation masque
    if mask is None:
        mask = np.zeros(pil.size[::-1], np.uint8)
    else:
        mask = np.array(mask, dtype=np.uint8)

    # Annotation finale
    if label == "authentic":
        annot = "authentic"
    else:
        annot = rle_encode((mask > 0).astype(np.uint8))

    rows.append({
        "case_id": Path(f).stem,
        "annotation": sanitize_annotation_str(annot),
        "area": int(dbg.get("area", mask.sum())),
        "mean": float(dbg.get("mean_inside", 0.0)),
        "thr": float(dbg.get("thr", 0.0))
    })


sub = pd.DataFrame(rows)
ss = pd.read_csv(SAMPLE_SUB)
ss["case_id"] = ss["case_id"].astype(str)
sub["case_id"] = sub["case_id"].astype(str)
final = ss[["case_id"]].merge(sub, on="case_id", how="left")
final["annotation"] = final["annotation"].fillna("authentic")
final[["case_id", "annotation"]].to_csv(OUT_PATH, index=False)

print(f"\n✅ Saved submission file: {OUT_PATH}")
print(final.head(10))


sample_files = sorted(os.listdir(TEST_DIR))[:5]
for f in sample_files:
    pil = Image.open(Path(TEST_DIR)/f).convert("RGB")
    label, mask, dbg = pipeline_final(pil)
    mask = np.array(mask, dtype=np.uint8) if mask is not None else np.zeros(pil.size[::-1], np.uint8)

    print(f"{'🔴' if label=='forged' else '🟢'} {f}: {label} | area={mask.sum()} mean={dbg.get('mean_inside', 0):.3f}")

    if label == "authentic":
        plt.figure(figsize=(5,5))
        plt.imshow(pil)
        plt.title(f"{f} — Authentic")
        plt.axis("off")
        plt.show()
    else:
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(pil)
        plt.title("Original Image")
        plt.axis("off")
        plt.subplot(1,2,2)
        plt.imshow(pil)
        plt.imshow(mask, alpha=0.45, cmap="Reds")
        plt.title(f"Predicted Forged Mask\nArea={mask.sum()} | Mean={dbg.get('mean_inside', 0):.3f}")
        plt.axis("off")
        plt.show()


In [ ]:
final["annotation"][0]

In [ ]:

def validate_annotation(annotation: str):
    # authentic はそのままでOK
    if annotation == "authentic":
        return None
    # セミコロン区切りで各要素が正しいJSON配列か検証し、rle_decodeできるかチェック
    parts = annotation.split(";")
    if any(p.strip() == "" for p in parts):
        return "Annotation contains empty part or trailing semicolon."
    try:
        # 各パーツは JSON の数列（start,length, start,length, ...）
        arrays = [np.asarray(json.loads(p), dtype=np.int32) for p in parts]
    except Exception:
        return "Annotation parts are not valid JSON arrays."
    # 昇順・重複なしなどの検証は rle_decode に委ねる
    try:
        label_rles = [rle_decode(x, shape=(720, 960)) for x in annotation.split(';')]
    except Exception:
        raise "decode error"
    # ここでは最小限のチェック（偶数長）
    for arr in arrays:
        if len(arr) % 2 != 0:
            return "RLE array length must be even (start/length pairs)."
        starts = arr[0::2]
        if sorted(starts.tolist()) != starts.tolist():
            return "RLE starts must be in ascending order."
    return None

In [ ]:
validate_annotation(final.annotation[0])

In [ ]:
label_rles = [rle_decode(x, shape=(720, 960)) for x in final.annotation[0].split(';')]

## supplemental debug

In [ ]:
def gt_masks_from_npy(mask_path: Path) -> list[np.ndarray]:
    m = np.load(mask_path)
    if m.ndim == 3:
        parts = [(m[k] > 0).astype(np.uint8) for k in range(m.shape[0])]
        return parts
    else:
        return [((m > 0).astype(np.uint8))]

# --- Validation using official oF1 on forged validation set ---
print("\n--- Official oF1 Validation ---")

SUPPLEMENT_DIR  = f"{BASE_DIR}/supplemental_images"
SUPPLE_MASK_DIR  = f"{BASE_DIR}/supplemental_masks"
supplemental_imgs = sorted([str(Path(SUPPLEMENT_DIR)/f) for f in os.listdir(SUPPLEMENT_DIR)])
supplemental_mask_imgs = sorted([str(Path(SUPPLE_MASK_DIR)/f) for f in os.listdir(SUPPLE_MASK_DIR)])

val_items = supplemental_imgs

scores = []

for p in tqdm(val_items, desc="oF1 forged-only"):

    pil = Image.open(p).convert("RGB")
    label, m_pred, dbg = pipeline_final(pil)

    # Ground-truth masks (instance-aware)
    gt_list = gt_masks_from_npy(Path(SUPPLE_MASK_DIR)/f"{Path(p).stem}.npy")

    # Predicted masks list: empty if classified authentic; else single mask
    if label == "authentic" or m_pred is None:
        pred_list = []
    else:
        pred = (np.array(m_pred) > 0).astype(np.uint8)
        pred_list = [pred]

    # Compute official oF1
    scores.append(oF1_score(pred_list, gt_list))

print(f"Average oF1 (Forged Validation) = {np.mean(scores):.4f}")

In [ ]:
gt_list